In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1,2'
from __future__ import division
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import scipy.io as sio
from scipy.linalg import qr
from skimage import draw
from joblib import Parallel, delayed
from utils import cpm_utils, tf_utils
from cpm_net import *
import cv2
import time
from numpy.linalg import inv
from scipy.spatial import distance
import math
import random
import sys
import scipy.io as sio

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
orig_h = 1080
orig_w = 1920
input_size = 368
hmap_size = 46
cmap_radius = 21
joints = 19
stages = 6

In [3]:
input_data = tf.placeholder(dtype=tf.float32, shape=[None, input_size, input_size, 3], name='input_image')

model = CPM_Model(stages, joints + 1)
model.build_model(input_data, 1)
saver = tf.train.Saver()


sess = tf.Session()
saver.restore(sess, "/media/yaoxx340/data/yaoxx340/cpm_ep/alg1.ckpt-1014")

INFO:tensorflow:Restoring parameters from /media/yaoxx340/data/yaoxx340/cpm_ep/alg1.ckpt-1014


In [4]:
bb = []
img = []
fv = []
vis = []
truth_x = []
truth_y = []

gt_content = open('/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/unlabel.txt', 'rb').readlines()

for idx, line in enumerate(gt_content):
    larger_h = 1
    line = line.split()
    cur_img_path = '/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/' + line[0]
    cur_img = cv2.imread(cur_img_path)
    print(cur_img_path)

    tmp = [float(x) for x in line[3:7]]
    frame_view = [int(x) for x in line[1:3]]
    cur_hand_bbox = [min([tmp[1], tmp[3]]), min([tmp[0], tmp[2]]),max([tmp[1], tmp[3]]),max([tmp[0], tmp[2]])]
    if cur_hand_bbox[0] < 0: cur_hand_bbox[0] = 0
    if cur_hand_bbox[1] < 0: cur_hand_bbox[1] = 0
    if cur_hand_bbox[2] > cur_img.shape[1]: cur_hand_bbox[2] = cur_img.shape[1]
    if cur_hand_bbox[3] > cur_img.shape[0]: cur_hand_bbox[3] = cur_img.shape[0]
    cur_hand_joints_y = [float(i) for i in line[7:60:2]]
    cur_hand_joints_x = [float(i) for i in line[8:60:2]]
    
    joint_vis = [cur_hand_joints_y[i] < orig_h and cur_hand_joints_x[i] < orig_w for i in range(len(cur_hand_joints_y))]
    vis.append(joint_vis)

    cur_img = cur_img[int(round(cur_hand_bbox[1])):int(round(cur_hand_bbox[3])),int(round(cur_hand_bbox[0])):int(round(cur_hand_bbox[2])),:]
    output_image = np.ones(shape=(input_size, input_size, 3)) * 128
    cur_hand_joints_x = [x - cur_hand_bbox[0] for x in cur_hand_joints_x]
    cur_hand_joints_y = [x - cur_hand_bbox[1] for x in cur_hand_joints_y]
    
    if cur_img.shape[0] > cur_img.shape[1]:
        scale = input_size / (cur_img.shape[0] * 1.0)
        hmap_scale = hmap_size / (cur_img.shape[0] * 1.0)
        cur_hand_joints_x = map(lambda x: x * hmap_scale, cur_hand_joints_x)
        cur_hand_joints_y = map(lambda x: x * hmap_scale, cur_hand_joints_y)

        image = cv2.resize(cur_img, (0, 0), fx=scale, fy=scale, interpolation=cv2.INTER_LANCZOS4)
        offset = image.shape[1] % 2
        hmap_image = cv2.resize(cur_img, (0, 0), fx=hmap_scale, fy=hmap_scale, interpolation=cv2.INTER_LANCZOS4)
        hmap_offset = hmap_image.shape[1] % 2

        output_image[:, int(input_size / 2 - math.floor(image.shape[1] / 2)): int(input_size / 2 + math.floor(image.shape[1] / 2) + offset), :] = image
        cur_hand_joints_x = map(lambda x: x + (hmap_size / 2 - math.floor(hmap_image.shape[1] / 2)),cur_hand_joints_x)
        fv.append((frame_view[0], frame_view[1]))
        img.append(output_image)
        truth_x.append(cur_hand_joints_x)
        truth_y.append(cur_hand_joints_y)
    
    
    else:
        scale = input_size / (cur_img.shape[1] * 1.0)
        hmap_scale = hmap_size / (cur_img.shape[1] * 1.0)
        cur_hand_joints_x = map(lambda x: x * hmap_scale, cur_hand_joints_x)
        cur_hand_joints_y = map(lambda x: x * hmap_scale, cur_hand_joints_y)
        
        image = cv2.resize(cur_img, (0, 0), fx=scale, fy=scale, interpolation=cv2.INTER_LANCZOS4)
        offset = image.shape[0] % 2
        hmap_image = cv2.resize(cur_img, (0, 0), fx=hmap_scale, fy=hmap_scale, interpolation=cv2.INTER_LANCZOS4)
        hmap_offset = hmap_image.shape[0] % 2

        output_image[int(input_size / 2 - math.floor(image.shape[0] / 2)): int(input_size / 2 + math.floor(image.shape[0] / 2) + offset), :, :] = image
        cur_hand_joints_y = map(lambda x: x + (hmap_size / 2 - math.floor(hmap_image.shape[0] / 2)),cur_hand_joints_y)
        fv.append((frame_view[0], frame_view[1]))
        img.append(output_image)
        truth_x.append(cur_hand_joints_x)
        truth_y.append(cur_hand_joints_y)        

/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00000211/00_00.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00000211/00_01.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00000211/00_02.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00000211/00_03.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00000211/00_04.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00000211/00_05.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00000211/00_06.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00000211/00_07.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00000211/00_08.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00000211/00_09.jpg
/media/yaoxx340/data/yaoxx340/

/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00000233/00_26.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00000233/00_27.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00000233/00_28.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00000233/00_29.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00000233/00_30.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00000244/00_00.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00000244/00_01.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00000244/00_02.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00000244/00_03.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00000244/00_04.jpg
/media/yaoxx340/data/yaoxx340/

/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00000266/00_21.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00000266/00_22.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00000266/00_23.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00000266/00_24.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00000266/00_25.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00000266/00_26.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00000266/00_27.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00000266/00_28.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00000266/00_29.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00000266/00_30.jpg
/media/yaoxx340/data/yaoxx340/

/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00000299/00_17.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00000299/00_18.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00000299/00_19.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00000299/00_20.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00000299/00_21.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00000299/00_22.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00000299/00_23.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00000299/00_24.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00000299/00_25.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00000299/00_26.jpg
/media/yaoxx340/data/yaoxx340/

/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00000332/00_10.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00000332/00_11.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00000332/00_12.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00000332/00_13.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00000332/00_14.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00000332/00_15.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00000332/00_16.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00000332/00_17.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00000332/00_18.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00000332/00_19.jpg
/media/yaoxx340/data/yaoxx340/

...

/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00005348/00_06.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00005348/00_07.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00005348/00_08.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00005348/00_09.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00005348/00_10.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00005348/00_11.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00005348/00_12.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00005348/00_13.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00005348/00_14.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00005348/00_15.jpg
/media/yaoxx340/data/yaoxx340/

/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00005381/00_00.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00005381/00_01.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00005381/00_02.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00005381/00_03.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00005381/00_04.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00005381/00_05.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00005381/00_06.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00005381/00_07.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00005381/00_08.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00005381/00_09.jpg
/media/yaoxx340/data/yaoxx340/

/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00005403/00_26.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00005403/00_27.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00005403/00_28.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00005403/00_29.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00005403/00_30.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00005414/00_00.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00005414/00_01.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00005414/00_02.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00005414/00_03.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00005414/00_04.jpg
/media/yaoxx340/data/yaoxx340/

/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00005447/00_21.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00005447/00_22.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00005447/00_23.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00005447/00_24.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00005447/00_25.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00005447/00_26.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00005447/00_27.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00005447/00_28.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00005447/00_29.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00005447/00_30.jpg
/media/yaoxx340/data/yaoxx340/

/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00005480/00_13.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00005480/00_14.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00005480/00_15.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00005480/00_16.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00005480/00_17.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00005480/00_18.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00005480/00_19.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00005480/00_20.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00005480/00_21.jpg
/media/yaoxx340/data/yaoxx340/panoptic-toolbox/scripts/171026_pose3/undis_img/00005480/00_22.jpg
/media/yaoxx340/data/yaoxx340/

In [5]:
split_img = [img[i:i + 40] for i in xrange(0, len(img), 40)]
predict_heatmap, stage_heatmap_np = sess.run([model.current_heatmap,
                                                              model.stage_heatmap,
                                                              ],
                                                             feed_dict={'input_image:0': [x / 255.0 for x in split_img[0]]})
for i in range(1, len(split_img)):
    predict_heatmap_2, stage_heatmap_np = sess.run([model.current_heatmap,
                                                              model.stage_heatmap,
                                                              ],
                                                             feed_dict={'input_image:0': [x / 255.0 for x in split_img[i]]})
    predict_heatmap = np.concatenate((predict_heatmap,predict_heatmap_2), axis=0)

In [6]:
def RestoreImageToOriginal(heat):
    max_coord = []
    max_value = []
    for joint_num in range(joints):
        joint_coord = np.unravel_index(np.argmax(heat[:, :, joint_num]),(heat.shape[0], heat.shape[1]))
        v = np.amax(heat[:, :, joint_num])
        max_coord.append([joint_coord[1], joint_coord[0]])
        max_value.append(v)
    return max_coord

In [7]:
maxPoints = [RestoreImageToOriginal(predict_heatmap[i,:,:,0:joints]) for i in range(0, predict_heatmap.shape[0])]

In [8]:
maxPoints = [item for sublist in maxPoints for item in sublist]
truth_x = [item for sublist in truth_x for item in sublist]
truth_y = [item for sublist in truth_y for item in sublist]

In [9]:
len(truth_y)

272118

In [10]:
truth_xy = [[truth_x[i], truth_y[i]] for i in range(len(truth_y))]
dst = [distance.euclidean(maxPoints[i],truth_xy[i]) for i in range(len(maxPoints))]

In [11]:
vis = [item for sublist in vis for item in sublist]

In [12]:
'''
filtered_dst_0 = np.array(dst[0::12])[np.array(vis[0::12])]
filtered_dst_1 = np.array(dst[1::12])[np.array(vis[1::12])]
filtered_dst_2 = np.array(dst[2::12])[np.array(vis[2::12])]
filtered_dst_3 = np.array(dst[3::12])[np.array(vis[3::12])]
filtered_dst_4 = np.array(dst[4::12])[np.array(vis[4::12])]
filtered_dst_5 = np.array(dst[5::12])[np.array(vis[5::12])]
filtered_dst_6 = np.array(dst[6::12])[np.array(vis[6::12])]
filtered_dst_7 = np.array(dst[7::12])[np.array(vis[7::12])]
filtered_dst_8 = np.array(dst[8::12])[np.array(vis[8::12])]
filtered_dst_9 = np.array(dst[9::12])[np.array(vis[9::12])]
filtered_dst_10 = np.array(dst[10::12])[np.array(vis[10::12])]
filtered_dst_11 = np.array(dst[11::12])[np.array(vis[11::12])]
#filtered_dst_12 = np.array(dst[12::12])[np.array(vis[12::12])]
'''
filtered_dst = np.array(dst)[np.array(vis)]

In [13]:
def drange(start, stop, step):
    r = start
    while r < stop:
        yield r
        r += step
def pck(thre, f_dst):
    return sum(i < thre for i in f_dst)/len(f_dst)

In [14]:
dis_range = drange(0.0, 40.0, 0.1)
dis_range = [ x for x in dis_range]

In [15]:
alg1 = [pck(i,filtered_dst) for i in dis_range]
'''
alg1_0 = [pck(i,filtered_dst_0) for i in dis_range]
alg1_1 = [pck(i,filtered_dst_1) for i in dis_range]
alg1_2 = [pck(i,filtered_dst_2) for i in dis_range]
alg1_3 = [pck(i,filtered_dst_3) for i in dis_range]
alg1_4 = [pck(i,filtered_dst_4) for i in dis_range]
alg1_5 = [pck(i,filtered_dst_5) for i in dis_range]
alg1_6 = [pck(i,filtered_dst_6) for i in dis_range]
alg1_7 = [pck(i,filtered_dst_7) for i in dis_range]
alg1_8 = [pck(i,filtered_dst_8) for i in dis_range]
alg1_9 = [pck(i,filtered_dst_9) for i in dis_range]
alg1_10 = [pck(i,filtered_dst_10) for i in dis_range]
alg1_11 = [pck(i,filtered_dst_11) for i in dis_range]
#alg1_12 = [pck(i,filtered_dst_12) for i in dis_range]
'''

'\nalg1_0 = [pck(i,filtered_dst_0) for i in dis_range]\nalg1_1 = [pck(i,filtered_dst_1) for i in dis_range]\nalg1_2 = [pck(i,filtered_dst_2) for i in dis_range]\nalg1_3 = [pck(i,filtered_dst_3) for i in dis_range]\nalg1_4 = [pck(i,filtered_dst_4) for i in dis_range]\nalg1_5 = [pck(i,filtered_dst_5) for i in dis_range]\nalg1_6 = [pck(i,filtered_dst_6) for i in dis_range]\nalg1_7 = [pck(i,filtered_dst_7) for i in dis_range]\nalg1_8 = [pck(i,filtered_dst_8) for i in dis_range]\nalg1_9 = [pck(i,filtered_dst_9) for i in dis_range]\nalg1_10 = [pck(i,filtered_dst_10) for i in dis_range]\nalg1_11 = [pck(i,filtered_dst_11) for i in dis_range]\n#alg1_12 = [pck(i,filtered_dst_12) for i in dis_range]\n'

In [16]:
sess.close()
sess = tf.Session()
saver.restore(sess, "/media/yaoxx340/data/yaoxx340/cpm_ep/alg2.ckpt-2093")

split_img = [img[i:i + 40] for i in xrange(0, len(img), 40)]
predict_heatmap, stage_heatmap_np = sess.run([model.current_heatmap,
                                                              model.stage_heatmap,
                                                              ],
                                                             feed_dict={'input_image:0': [x / 255.0 for x in split_img[0]]})
for i in range(1, len(split_img)):
    predict_heatmap_2, stage_heatmap_np = sess.run([model.current_heatmap,
                                                              model.stage_heatmap,
                                                              ],
                                                             feed_dict={'input_image:0': [x / 255.0 for x in split_img[i]]})
    predict_heatmap = np.concatenate((predict_heatmap,predict_heatmap_2), axis=0)
    
maxPoints = [RestoreImageToOriginal(predict_heatmap[i,:,:,0:joints]) for i in range(0, predict_heatmap.shape[0])]
maxPoints = [item for sublist in maxPoints for item in sublist]
dst = [distance.euclidean(maxPoints[i],truth_xy[i]) for i in range(len(maxPoints))]
'''
filtered_dst_0 = np.array(dst[0::12])[np.array(vis[0::12])]
filtered_dst_1 = np.array(dst[1::12])[np.array(vis[1::12])]
filtered_dst_2 = np.array(dst[2::12])[np.array(vis[2::12])]
filtered_dst_3 = np.array(dst[3::12])[np.array(vis[3::12])]
filtered_dst_4 = np.array(dst[4::12])[np.array(vis[4::12])]
filtered_dst_5 = np.array(dst[5::12])[np.array(vis[5::12])]
filtered_dst_6 = np.array(dst[6::12])[np.array(vis[6::12])]
filtered_dst_7 = np.array(dst[7::12])[np.array(vis[7::12])]
filtered_dst_8 = np.array(dst[8::12])[np.array(vis[8::12])]
filtered_dst_9 = np.array(dst[9::12])[np.array(vis[9::12])]
filtered_dst_10 = np.array(dst[10::12])[np.array(vis[10::12])]
filtered_dst_11 = np.array(dst[11::12])[np.array(vis[11::12])]
#filtered_dst_12 = np.array(dst[12::12])[np.array(vis[12::12])]
'''
filtered_dst = np.array(dst)[np.array(vis)]

alg2 = [pck(i,filtered_dst) for i in dis_range]
'''
alg2_0 = [pck(i,filtered_dst_0) for i in dis_range]
alg2_1 = [pck(i,filtered_dst_1) for i in dis_range]
alg2_2 = [pck(i,filtered_dst_2) for i in dis_range]
alg2_3 = [pck(i,filtered_dst_3) for i in dis_range]
alg2_4 = [pck(i,filtered_dst_4) for i in dis_range]
alg2_5 = [pck(i,filtered_dst_5) for i in dis_range]
alg2_6 = [pck(i,filtered_dst_6) for i in dis_range]
alg2_7 = [pck(i,filtered_dst_7) for i in dis_range]
alg2_8 = [pck(i,filtered_dst_8) for i in dis_range]
alg2_9 = [pck(i,filtered_dst_9) for i in dis_range]
alg2_10 = [pck(i,filtered_dst_10) for i in dis_range]
alg2_11 = [pck(i,filtered_dst_11) for i in dis_range]
#alg2_12 = [pck(i,filtered_dst_12) for i in dis_range]
'''

INFO:tensorflow:Restoring parameters from /media/yaoxx340/data/yaoxx340/cpm_ep/alg2.ckpt-2093


'\nalg2_0 = [pck(i,filtered_dst_0) for i in dis_range]\nalg2_1 = [pck(i,filtered_dst_1) for i in dis_range]\nalg2_2 = [pck(i,filtered_dst_2) for i in dis_range]\nalg2_3 = [pck(i,filtered_dst_3) for i in dis_range]\nalg2_4 = [pck(i,filtered_dst_4) for i in dis_range]\nalg2_5 = [pck(i,filtered_dst_5) for i in dis_range]\nalg2_6 = [pck(i,filtered_dst_6) for i in dis_range]\nalg2_7 = [pck(i,filtered_dst_7) for i in dis_range]\nalg2_8 = [pck(i,filtered_dst_8) for i in dis_range]\nalg2_9 = [pck(i,filtered_dst_9) for i in dis_range]\nalg2_10 = [pck(i,filtered_dst_10) for i in dis_range]\nalg2_11 = [pck(i,filtered_dst_11) for i in dis_range]\n#alg2_12 = [pck(i,filtered_dst_12) for i in dis_range]\n'

In [17]:
sess.close()
sess = tf.Session()
saver.restore(sess, "/media/yaoxx340/data/yaoxx340/cpm_ep/alg3_iter1.ckpt-1698")

split_img = [img[i:i + 40] for i in xrange(0, len(img), 40)]
predict_heatmap, stage_heatmap_np = sess.run([model.current_heatmap,
                                                              model.stage_heatmap,
                                                              ],
                                                             feed_dict={'input_image:0': [x / 255.0 for x in split_img[0]]})
for i in range(1, len(split_img)):
    predict_heatmap_2, stage_heatmap_np = sess.run([model.current_heatmap,
                                                              model.stage_heatmap,
                                                              ],
                                                             feed_dict={'input_image:0': [x / 255.0 for x in split_img[i]]})
    predict_heatmap = np.concatenate((predict_heatmap,predict_heatmap_2), axis=0)
    
maxPoints = [RestoreImageToOriginal(predict_heatmap[i,:,:,0:joints]) for i in range(0, predict_heatmap.shape[0])]
maxPoints = [item for sublist in maxPoints for item in sublist]
dst = [distance.euclidean(maxPoints[i],truth_xy[i]) for i in range(len(maxPoints))]
'''
filtered_dst_0 = np.array(dst[0::12])[np.array(vis[0::12])]
filtered_dst_1 = np.array(dst[1::12])[np.array(vis[1::12])]
filtered_dst_2 = np.array(dst[2::12])[np.array(vis[2::12])]
filtered_dst_3 = np.array(dst[3::12])[np.array(vis[3::12])]
filtered_dst_4 = np.array(dst[4::12])[np.array(vis[4::12])]
filtered_dst_5 = np.array(dst[5::12])[np.array(vis[5::12])]
filtered_dst_6 = np.array(dst[6::12])[np.array(vis[6::12])]
filtered_dst_7 = np.array(dst[7::12])[np.array(vis[7::12])]
filtered_dst_8 = np.array(dst[8::12])[np.array(vis[8::12])]
filtered_dst_9 = np.array(dst[9::12])[np.array(vis[9::12])]
filtered_dst_10 = np.array(dst[10::12])[np.array(vis[10::12])]
filtered_dst_11 = np.array(dst[11::12])[np.array(vis[11::12])]
#filtered_dst_12 = np.array(dst[12::12])[np.array(vis[12::12])]
'''
filtered_dst = np.array(dst)[np.array(vis)]

alg3 = [pck(i,filtered_dst) for i in dis_range]
'''
alg3_0 = [pck(i,filtered_dst_0) for i in dis_range]
alg3_1 = [pck(i,filtered_dst_1) for i in dis_range]
alg3_2 = [pck(i,filtered_dst_2) for i in dis_range]
alg3_3 = [pck(i,filtered_dst_3) for i in dis_range]
alg3_4 = [pck(i,filtered_dst_4) for i in dis_range]
alg3_5 = [pck(i,filtered_dst_5) for i in dis_range]
alg3_6 = [pck(i,filtered_dst_6) for i in dis_range]
alg3_7 = [pck(i,filtered_dst_7) for i in dis_range]
alg3_8 = [pck(i,filtered_dst_8) for i in dis_range]
alg3_9 = [pck(i,filtered_dst_9) for i in dis_range]
alg3_10 = [pck(i,filtered_dst_10) for i in dis_range]
alg3_11 = [pck(i,filtered_dst_11) for i in dis_range]
#alg3_12 = [pck(i,filtered_dst_12) for i in dis_range]
'''

INFO:tensorflow:Restoring parameters from /media/yaoxx340/data/yaoxx340/cpm_ep/alg3_iter1.ckpt-1698


'\nalg3_0 = [pck(i,filtered_dst_0) for i in dis_range]\nalg3_1 = [pck(i,filtered_dst_1) for i in dis_range]\nalg3_2 = [pck(i,filtered_dst_2) for i in dis_range]\nalg3_3 = [pck(i,filtered_dst_3) for i in dis_range]\nalg3_4 = [pck(i,filtered_dst_4) for i in dis_range]\nalg3_5 = [pck(i,filtered_dst_5) for i in dis_range]\nalg3_6 = [pck(i,filtered_dst_6) for i in dis_range]\nalg3_7 = [pck(i,filtered_dst_7) for i in dis_range]\nalg3_8 = [pck(i,filtered_dst_8) for i in dis_range]\nalg3_9 = [pck(i,filtered_dst_9) for i in dis_range]\nalg3_10 = [pck(i,filtered_dst_10) for i in dis_range]\nalg3_11 = [pck(i,filtered_dst_11) for i in dis_range]\n#alg3_12 = [pck(i,filtered_dst_12) for i in dis_range]\n'

In [18]:
sess.close()
sess = tf.Session()
saver.restore(sess, "/media/yaoxx340/data/yaoxx340/cpm_ep/alg3_iter2.ckpt-595")

split_img = [img[i:i + 40] for i in xrange(0, len(img), 40)]
predict_heatmap, stage_heatmap_np = sess.run([model.current_heatmap,
                                                              model.stage_heatmap,
                                                              ],
                                                             feed_dict={'input_image:0': [x / 255.0 for x in split_img[0]]})
for i in range(1, len(split_img)):
    predict_heatmap_2, stage_heatmap_np = sess.run([model.current_heatmap,
                                                              model.stage_heatmap,
                                                              ],
                                                             feed_dict={'input_image:0': [x / 255.0 for x in split_img[i]]})
    predict_heatmap = np.concatenate((predict_heatmap,predict_heatmap_2), axis=0)
    
maxPoints = [RestoreImageToOriginal(predict_heatmap[i,:,:,0:joints]) for i in range(0, predict_heatmap.shape[0])]
maxPoints = [item for sublist in maxPoints for item in sublist]
dst = [distance.euclidean(maxPoints[i],truth_xy[i]) for i in range(len(maxPoints))]
'''
filtered_dst_0 = np.array(dst[0::12])[np.array(vis[0::12])]
filtered_dst_1 = np.array(dst[1::12])[np.array(vis[1::12])]
filtered_dst_2 = np.array(dst[2::12])[np.array(vis[2::12])]
filtered_dst_3 = np.array(dst[3::12])[np.array(vis[3::12])]
filtered_dst_4 = np.array(dst[4::12])[np.array(vis[4::12])]
filtered_dst_5 = np.array(dst[5::12])[np.array(vis[5::12])]
filtered_dst_6 = np.array(dst[6::12])[np.array(vis[6::12])]
filtered_dst_7 = np.array(dst[7::12])[np.array(vis[7::12])]
filtered_dst_8 = np.array(dst[8::12])[np.array(vis[8::12])]
filtered_dst_9 = np.array(dst[9::12])[np.array(vis[9::12])]
filtered_dst_10 = np.array(dst[10::12])[np.array(vis[10::12])]
filtered_dst_11 = np.array(dst[11::12])[np.array(vis[11::12])]
#filtered_dst_12 = np.array(dst[12::12])[np.array(vis[12::12])]
'''
filtered_dst = np.array(dst)[np.array(vis)]

alg4 = [pck(i,filtered_dst) for i in dis_range]
'''
alg4_0 = [pck(i,filtered_dst_0) for i in dis_range]
alg4_1 = [pck(i,filtered_dst_1) for i in dis_range]
alg4_2 = [pck(i,filtered_dst_2) for i in dis_range]
alg4_3 = [pck(i,filtered_dst_3) for i in dis_range]
alg4_4 = [pck(i,filtered_dst_4) for i in dis_range]
alg4_5 = [pck(i,filtered_dst_5) for i in dis_range]
alg4_6 = [pck(i,filtered_dst_6) for i in dis_range]
alg4_7 = [pck(i,filtered_dst_7) for i in dis_range]
alg4_8 = [pck(i,filtered_dst_8) for i in dis_range]
alg4_9 = [pck(i,filtered_dst_9) for i in dis_range]
alg4_10 = [pck(i,filtered_dst_10) for i in dis_range]
alg4_11 = [pck(i,filtered_dst_11) for i in dis_range]
#alg4_12 = [pck(i,filtered_dst_12) for i in dis_range]
'''

INFO:tensorflow:Restoring parameters from /media/yaoxx340/data/yaoxx340/cpm_ep/alg3_iter2.ckpt-595


'\nalg4_0 = [pck(i,filtered_dst_0) for i in dis_range]\nalg4_1 = [pck(i,filtered_dst_1) for i in dis_range]\nalg4_2 = [pck(i,filtered_dst_2) for i in dis_range]\nalg4_3 = [pck(i,filtered_dst_3) for i in dis_range]\nalg4_4 = [pck(i,filtered_dst_4) for i in dis_range]\nalg4_5 = [pck(i,filtered_dst_5) for i in dis_range]\nalg4_6 = [pck(i,filtered_dst_6) for i in dis_range]\nalg4_7 = [pck(i,filtered_dst_7) for i in dis_range]\nalg4_8 = [pck(i,filtered_dst_8) for i in dis_range]\nalg4_9 = [pck(i,filtered_dst_9) for i in dis_range]\nalg4_10 = [pck(i,filtered_dst_10) for i in dis_range]\nalg4_11 = [pck(i,filtered_dst_11) for i in dis_range]\n#alg4_12 = [pck(i,filtered_dst_12) for i in dis_range]\n'

In [39]:
#sess.close()
sess = tf.Session()
saver.restore(sess, "/media/yaoxx340/data/yaoxx340/EpiNet/model/alg5.ckpt-3501")

split_img = [img[i:i + 40] for i in xrange(0, len(img), 40)]
predict_heatmap, stage_heatmap_np = sess.run([model.current_heatmap,
                                                              model.stage_heatmap,
                                                              ],
                                                             feed_dict={'input_image:0': [x / 255.0 for x in split_img[0]]})
for i in range(1, len(split_img)):
    predict_heatmap_2, stage_heatmap_np = sess.run([model.current_heatmap,
                                                              model.stage_heatmap,
                                                              ],
                                                             feed_dict={'input_image:0': [x / 255.0 for x in split_img[i]]})
    predict_heatmap = np.concatenate((predict_heatmap,predict_heatmap_2), axis=0)


    
maxPoints = [RestoreImageToOriginal(predict_heatmap[i,:,:,0:joints]) for i in range(0, predict_heatmap.shape[0])]
maxPoints = [item for sublist in maxPoints for item in sublist]
dst = [distance.euclidean(maxPoints[i],truth_xy[i]) for i in range(len(maxPoints))]
'''
filtered_dst_0 = np.array(dst[0::12])[np.array(vis[0::12])]
filtered_dst_1 = np.array(dst[1::12])[np.array(vis[1::12])]
filtered_dst_2 = np.array(dst[2::12])[np.array(vis[2::12])]
filtered_dst_3 = np.array(dst[3::12])[np.array(vis[3::12])]
filtered_dst_4 = np.array(dst[4::12])[np.array(vis[4::12])]
filtered_dst_5 = np.array(dst[5::12])[np.array(vis[5::12])]
filtered_dst_6 = np.array(dst[6::12])[np.array(vis[6::12])]
filtered_dst_7 = np.array(dst[7::12])[np.array(vis[7::12])]
filtered_dst_8 = np.array(dst[8::12])[np.array(vis[8::12])]
filtered_dst_9 = np.array(dst[9::12])[np.array(vis[9::12])]
filtered_dst_10 = np.array(dst[10::12])[np.array(vis[10::12])]
filtered_dst_11 = np.array(dst[11::12])[np.array(vis[11::12])]
#filtered_dst_12 = np.array(dst[12::12])[np.array(vis[12::12])]
'''
filtered_dst = np.array(dst)[np.array(vis)]

alg4_iter2 = [pck(i,filtered_dst) for i in dis_range]
'''
alg4_0_iter2 = [pck(i,filtered_dst_0) for i in dis_range]
alg4_1_iter2 = [pck(i,filtered_dst_1) for i in dis_range]
alg4_2_iter2 = [pck(i,filtered_dst_2) for i in dis_range]
alg4_3_iter2 = [pck(i,filtered_dst_3) for i in dis_range]
alg4_4_iter2 = [pck(i,filtered_dst_4) for i in dis_range]
alg4_5_iter2 = [pck(i,filtered_dst_5) for i in dis_range]
alg4_6_iter2 = [pck(i,filtered_dst_6) for i in dis_range]
alg4_7_iter2 = [pck(i,filtered_dst_7) for i in dis_range]
alg4_8_iter2 = [pck(i,filtered_dst_8) for i in dis_range]
alg4_9_iter2 = [pck(i,filtered_dst_9) for i in dis_range]
alg4_10_iter2 = [pck(i,filtered_dst_10) for i in dis_range]
alg4_11_iter2 = [pck(i,filtered_dst_11) for i in dis_range]
#alg4_12_iter2 = [pck(i,filtered_dst_12) for i in dis_range]
'''

INFO:tensorflow:Restoring parameters from /media/yaoxx340/data/yaoxx340/EpiNet/model/alg5.ckpt-3501


'\nalg4_0_iter2 = [pck(i,filtered_dst_0) for i in dis_range]\nalg4_1_iter2 = [pck(i,filtered_dst_1) for i in dis_range]\nalg4_2_iter2 = [pck(i,filtered_dst_2) for i in dis_range]\nalg4_3_iter2 = [pck(i,filtered_dst_3) for i in dis_range]\nalg4_4_iter2 = [pck(i,filtered_dst_4) for i in dis_range]\nalg4_5_iter2 = [pck(i,filtered_dst_5) for i in dis_range]\nalg4_6_iter2 = [pck(i,filtered_dst_6) for i in dis_range]\nalg4_7_iter2 = [pck(i,filtered_dst_7) for i in dis_range]\nalg4_8_iter2 = [pck(i,filtered_dst_8) for i in dis_range]\nalg4_9_iter2 = [pck(i,filtered_dst_9) for i in dis_range]\nalg4_10_iter2 = [pck(i,filtered_dst_10) for i in dis_range]\nalg4_11_iter2 = [pck(i,filtered_dst_11) for i in dis_range]\n#alg4_12_iter2 = [pck(i,filtered_dst_12) for i in dis_range]\n'

In [40]:
'''
sess.close()
sess = tf.Session()
saver.restore(sess, "/media/yaoxx340/data/yaoxx340/EpiNet/dog/alg5.ckpt-13001")

split_img = [img[i:i + 40] for i in xrange(0, len(img), 40)]
predict_heatmap, stage_heatmap_np = sess.run([model.current_heatmap,
                                                              model.stage_heatmap,
                                                              ],
                                                             feed_dict={'input_image:0': [x / 255.0 for x in split_img[0]]})
for i in range(1, len(split_img)):
    predict_heatmap_2, stage_heatmap_np = sess.run([model.current_heatmap,
                                                              model.stage_heatmap,
                                                              ],
                                                             feed_dict={'input_image:0': [x / 255.0 for x in split_img[i]]})
    predict_heatmap = np.concatenate((predict_heatmap,predict_heatmap_2), axis=0)


    
maxPoints = [RestoreImageToOriginal(predict_heatmap[i,:,:,0:joints]) for i in range(0, predict_heatmap.shape[0])]
maxPoints = [item for sublist in maxPoints for item in sublist]
dst = [distance.euclidean(maxPoints[i],truth_xy[i]) for i in range(len(maxPoints))]
filtered_dst_0 = np.array(dst[0::12])[np.array(vis[0::12])]
filtered_dst_1 = np.array(dst[1::12])[np.array(vis[1::12])]
filtered_dst_2 = np.array(dst[2::12])[np.array(vis[2::12])]
filtered_dst_3 = np.array(dst[3::12])[np.array(vis[3::12])]
filtered_dst_4 = np.array(dst[4::12])[np.array(vis[4::12])]
filtered_dst_5 = np.array(dst[5::12])[np.array(vis[5::12])]
filtered_dst_6 = np.array(dst[6::12])[np.array(vis[6::12])]
filtered_dst_7 = np.array(dst[7::12])[np.array(vis[7::12])]
filtered_dst_8 = np.array(dst[8::12])[np.array(vis[8::12])]
filtered_dst_9 = np.array(dst[9::12])[np.array(vis[9::12])]
filtered_dst_10 = np.array(dst[10::12])[np.array(vis[10::12])]
filtered_dst_11 = np.array(dst[11::12])[np.array(vis[11::12])]
#filtered_dst_12 = np.array(dst[12::12])[np.array(vis[12::12])]
filtered_dst = np.array(dst)[np.array(vis)]

alg5 = [pck(i,filtered_dst) for i in dis_range]
alg5_0 = [pck(i,filtered_dst_0) for i in dis_range]
alg5_1 = [pck(i,filtered_dst_1) for i in dis_range]
alg5_2 = [pck(i,filtered_dst_2) for i in dis_range]
alg5_3 = [pck(i,filtered_dst_3) for i in dis_range]
alg5_4 = [pck(i,filtered_dst_4) for i in dis_range]
alg5_5 = [pck(i,filtered_dst_5) for i in dis_range]
alg5_6 = [pck(i,filtered_dst_6) for i in dis_range]
alg5_7 = [pck(i,filtered_dst_7) for i in dis_range]
alg5_8 = [pck(i,filtered_dst_8) for i in dis_range]
alg5_9 = [pck(i,filtered_dst_9) for i in dis_range]
alg5_10 = [pck(i,filtered_dst_10) for i in dis_range]
alg5_11 = [pck(i,filtered_dst_11) for i in dis_range]
#alg5_12 = [pck(i,filtered_dst_12) for i in dis_range]
'''

'\nsess.close()\nsess = tf.Session()\nsaver.restore(sess, "/media/yaoxx340/data/yaoxx340/EpiNet/dog/alg5.ckpt-13001")\n\nsplit_img = [img[i:i + 40] for i in xrange(0, len(img), 40)]\npredict_heatmap, stage_heatmap_np = sess.run([model.current_heatmap,\n                                                              model.stage_heatmap,\n                                                              ],\n                                                             feed_dict={\'input_image:0\': [x / 255.0 for x in split_img[0]]})\nfor i in range(1, len(split_img)):\n    predict_heatmap_2, stage_heatmap_np = sess.run([model.current_heatmap,\n                                                              model.stage_heatmap,\n                                                              ],\n                                                             feed_dict={\'input_image:0\': [x / 255.0 for x in split_img[i]]})\n    predict_heatmap = np.concatenate((predict_heatmap,predict_heatmap_2), axis=

In [41]:
'''
plt.rcParams.update({'font.size': 22})
f = plt.figure(figsize=(300,20))

plt.subplot(1,14,1)
plt.plot(dis_range, alg1)
plt.plot(dis_range, alg2)
plt.plot(dis_range, alg3)
plt.plot(dis_range, alg4)
plt.plot(dis_range, alg4_iter2)
plt.plot(dis_range, alg5)
plt.legend(['labeled', 'spatial', 'spatio-temporal', 'RANSAC (iter1)', 'RANSAC (iter2)', 'EpiNet'], loc='lower right')
plt.title('PCK: Overall')
plt.xlabel('Distance')
plt.ylabel('Detection rate')

plt.subplot(1,14,2)
plt.plot(dis_range, alg1_0)
plt.plot(dis_range, alg2_0)
plt.plot(dis_range, alg3_0)
plt.plot(dis_range, alg4_0)
plt.plot(dis_range, alg4_0_iter2)
plt.plot(dis_range, alg5_0)
plt.legend(['labeled', 'spatial', 'spatio-temporal', 'RANSAC (iter1)', 'RANSAC (iter2)', 'EpiNet'], loc='lower right')
plt.title('PCK: Neck')
plt.xlabel('Distance')
plt.ylabel('Detection rate')

plt.subplot(1,14,3)
plt.plot(dis_range, alg1_1)
plt.plot(dis_range, alg2_1)
plt.plot(dis_range, alg3_1)
plt.plot(dis_range, alg4_1)
plt.plot(dis_range, alg4_1_iter2)
plt.plot(dis_range, alg5_1)
plt.legend(['labeled', 'spatial', 'spatio-temporal', 'RANSAC (iter1)', 'RANSAC (iter2)', 'EpiNet'], loc='lower right')
plt.title('PCK: Right Shoulder')
plt.xlabel('Distance')
plt.ylabel('Detection rate')

plt.subplot(1,14,4)
plt.plot(dis_range, alg1_2)
plt.plot(dis_range, alg2_2)
plt.plot(dis_range, alg3_2)
plt.plot(dis_range, alg4_2)
plt.plot(dis_range, alg4_2_iter2)
plt.plot(dis_range, alg5_2)
plt.legend(['labeled', 'spatial', 'spatio-temporal', 'RANSAC (iter1)', 'RANSAC (iter2)', 'EpiNet'], loc='lower right')
plt.title('PCK: Right Elbow')
plt.xlabel('Distance')
plt.ylabel('Detection rate')

plt.subplot(1,14,5)
plt.plot(dis_range, alg1_3)
plt.plot(dis_range, alg2_3)
plt.plot(dis_range, alg3_3)
plt.plot(dis_range, alg4_3)
plt.plot(dis_range, alg4_3_iter2)
plt.plot(dis_range, alg5_3)
plt.legend(['labeled', 'spatial', 'spatio-temporal', 'RANSAC (iter1)', 'RANSAC (iter2)', 'EpiNet'], loc='lower right')
plt.title('PCK: Right Wrist')
plt.xlabel('Distance')
plt.ylabel('Detection rate')

plt.subplot(1,14,6)
plt.plot(dis_range, alg1_4)
plt.plot(dis_range, alg2_4)
plt.plot(dis_range, alg3_4)
plt.plot(dis_range, alg4_4)
plt.plot(dis_range, alg4_4_iter2)
plt.plot(dis_range, alg5_4)
plt.legend(['labeled', 'spatial', 'spatio-temporal', 'RANSAC (iter1)', 'RANSAC (iter2)', 'EpiNet'], loc='lower right')
plt.title('PCK: Left Shoulder')
plt.xlabel('Distance')
plt.ylabel('Detection rate')

plt.subplot(1,14,7)
plt.plot(dis_range, alg1_5)
plt.plot(dis_range, alg2_5)
plt.plot(dis_range, alg3_5)
plt.plot(dis_range, alg4_5)
plt.plot(dis_range, alg4_5_iter2)
plt.plot(dis_range, alg5_5)
plt.legend(['labeled', 'spatial', 'spatio-temporal', 'RANSAC (iter1)', 'RANSAC (iter2)', 'EpiNet'], loc='lower right')
plt.title('PCK: Left Elbow')
plt.xlabel('Distance')
plt.ylabel('Detection rate')

plt.subplot(1,14,8)
plt.plot(dis_range, alg1_6)
plt.plot(dis_range, alg2_6)
plt.plot(dis_range, alg3_6)
plt.plot(dis_range, alg4_6)
plt.plot(dis_range, alg4_6_iter2)
plt.plot(dis_range, alg5_6)
plt.legend(['labeled', 'spatial', 'spatio-temporal', 'RANSAC (iter1)', 'RANSAC (iter2)', 'EpiNet'], loc='lower right')
plt.title('PCK: Left Wrist')
plt.xlabel('Distance')
plt.ylabel('Detection rate')

plt.subplot(1,14,9)
plt.plot(dis_range, alg1_7)
plt.plot(dis_range, alg2_7)
plt.plot(dis_range, alg3_7)
plt.plot(dis_range, alg4_7)
plt.plot(dis_range, alg4_7_iter2)
plt.plot(dis_range, alg5_7)
plt.legend(['labeled', 'spatial', 'spatio-temporal', 'RANSAC (iter1)', 'RANSAC (iter2)', 'EpiNet'], loc='lower right')
plt.title('PCK: Right Hip')
plt.xlabel('Distance')
plt.ylabel('Detection rate')

plt.subplot(1,14,10)
plt.plot(dis_range, alg1_8)
plt.plot(dis_range, alg2_8)
plt.plot(dis_range, alg3_8)
plt.plot(dis_range, alg4_8)
plt.plot(dis_range, alg4_8_iter2)
plt.plot(dis_range, alg5_8)
plt.legend(['labeled', 'spatial', 'spatio-temporal', 'RANSAC (iter1)', 'RANSAC (iter2)', 'EpiNet'], loc='lower right')
plt.title('PCK: Right Knee')
plt.xlabel('Distance')
plt.ylabel('Detection rate')

plt.subplot(1,14,11)
plt.plot(dis_range, alg1_9)
plt.plot(dis_range, alg2_9)
plt.plot(dis_range, alg3_9)
plt.plot(dis_range, alg4_9)
plt.plot(dis_range, alg4_9_iter2)
plt.plot(dis_range, alg5_9)
plt.legend(['labeled', 'spatial', 'spatio-temporal', 'RANSAC (iter1)', 'RANSAC (iter2)', 'EpiNet'], loc='lower right')
plt.title('PCK: Right Ankle')
plt.xlabel('Distance')
plt.ylabel('Detection rate')

plt.subplot(1,14,12)
plt.plot(dis_range, alg1_10)
plt.plot(dis_range, alg2_10)
plt.plot(dis_range, alg3_10)
plt.plot(dis_range, alg4_10)
plt.plot(dis_range, alg4_10_iter2)
plt.plot(dis_range, alg5_10)
plt.legend(['labeled', 'spatial', 'spatio-temporal', 'RANSAC (iter1)', 'RANSAC (iter2)', 'EpiNet'], loc='lower right')
plt.title('PCK: Left Hip')
plt.xlabel('Distance')
plt.ylabel('Detection rate')

plt.subplot(1,14,13)
plt.plot(dis_range, alg1_11)
plt.plot(dis_range, alg2_11)
plt.plot(dis_range, alg3_11)
plt.plot(dis_range, alg4_11)
plt.plot(dis_range, alg4_11_iter2)
plt.plot(dis_range, alg5_11)
plt.legend(['labeled', 'spatial', 'spatio-temporal', 'RANSAC (iter1)', 'RANSAC (iter2)', 'EpiNet'], loc='lower right')
plt.title('PCK: Left Knee')
plt.xlabel('Distance')
plt.ylabel('Detection rate')

plt.subplot(1,14,14)
plt.plot(dis_range, alg1_11)
plt.plot(dis_range, alg2_11)
plt.plot(dis_range, alg3_11)
plt.plot(dis_range, alg4_11)
plt.plot(dis_range, alg4_11_iter2)
plt.plot(dis_range, alg5_11)
plt.legend(['labeled', 'spatial', 'spatio-temporal', 'RANSAC (iter1)', 'RANSAC (iter2)', 'EpiNet'], loc='lower right')
plt.title('PCK: Left Ankle')
plt.xlabel('Distance')
plt.ylabel('Detection rate')

#plt.subplots_adjust(bottom=0.15, wspace=0.1, top=0.15)

plt.show()

#f.savefig("pck_js.pdf", bbox_inches='tight')
'''

'\nplt.rcParams.update({\'font.size\': 22})\nf = plt.figure(figsize=(300,20))\n\nplt.subplot(1,14,1)\nplt.plot(dis_range, alg1)\nplt.plot(dis_range, alg2)\nplt.plot(dis_range, alg3)\nplt.plot(dis_range, alg4)\nplt.plot(dis_range, alg4_iter2)\nplt.plot(dis_range, alg5)\nplt.legend([\'labeled\', \'spatial\', \'spatio-temporal\', \'RANSAC (iter1)\', \'RANSAC (iter2)\', \'EpiNet\'], loc=\'lower right\')\nplt.title(\'PCK: Overall\')\nplt.xlabel(\'Distance\')\nplt.ylabel(\'Detection rate\')\n\nplt.subplot(1,14,2)\nplt.plot(dis_range, alg1_0)\nplt.plot(dis_range, alg2_0)\nplt.plot(dis_range, alg3_0)\nplt.plot(dis_range, alg4_0)\nplt.plot(dis_range, alg4_0_iter2)\nplt.plot(dis_range, alg5_0)\nplt.legend([\'labeled\', \'spatial\', \'spatio-temporal\', \'RANSAC (iter1)\', \'RANSAC (iter2)\', \'EpiNet\'], loc=\'lower right\')\nplt.title(\'PCK: Neck\')\nplt.xlabel(\'Distance\')\nplt.ylabel(\'Detection rate\')\n\nplt.subplot(1,14,3)\nplt.plot(dis_range, alg1_1)\nplt.plot(dis_range, alg2_1)\nplt.plo

In [42]:
sess.close()

In [43]:
result = [dis_range, alg1, alg2, alg3, alg4, alg4_iter2]
sio.savemat('/media/yaoxx340/data/yaoxx340/vis/overall.mat', {'vect':np.array(result)})